In [32]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [33]:
from imports import *
from functions import *

Connection to PostgreSQL

In [34]:
path_postgresql_creds = r"C:\Users\f.gionnane\Documents\Data Engineering\Credentials\postgresql_creds.json"

with open(path_postgresql_creds, 'r') as file:
    content = json.load(file)
    user = content["user"]
    password = content["password"]
    host = content["host"]
    port = content["port"]

db = "Oceanography_ML_Project"
schema_bronze = "Bronze"
schema_silver = "Silver"

# Créer l'engine PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}")
conn = engine.connect()

Charger les Données des Tables de la couche de Bronze

In [35]:
# Charger les métadonnées du schéma existant
metadata = MetaData(schema=schema_bronze)
print("\n🔍 Chargement des métadonnées du schéma...")
metadata.reflect(bind=conn)
print("✅ Métadonnées chargées avec succès.\n")

# Récupérer les noms des tables
table_names = [t.name for t in metadata.sorted_tables]
print(f"🔢 Nombre total de tables dans le schéma : {len(table_names)}\n")

# Filtrer les tables en fonction du contenu de leur nom
marine_tables = {t for t in table_names if "marine" in t.lower()}
meteo_tables = {t for t in table_names if "meteo" in t.lower()}
buoys_data_table = {t for t in table_names if "buoy" in t.lower()}

print(f"🌊 Tables marines trouvées : {len(marine_tables)}")
print(f"🌧️ Tables météo trouvées : {len(meteo_tables)}")
print(f"🐋 Tables de bouées trouvées : {len(buoys_data_table)}\n")

# Initialiser le dictionnaire des résultats
buoys_datas = {}

# Compteurs pour suivre le nombre de tables chargées avec succès
marine_data_count = 0
meteo_data_count = 0
buoys_data_count = 0

# Compteur pour le nombre total de lignes
total_marine_rows = 0
total_meteo_rows = 0
total_buoys_rows = 0  # Changer ici pour compter le nombre de lignes (bouées)

# Vérifier et récupérer les données de la table "buoys_datas"
if buoys_data_table:
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    print("🔄 Chargement des données de la table 'buoys_datas'...")
    buoy_data_table_name = next(iter(buoys_data_table))

    try:
        buoys_datas_raw = fetch_table_data(schema=schema_bronze, conn=conn, table_name=buoy_data_table_name, as_df=True)

        if buoys_datas_raw is not None:
            print("📦 Données récupérées pour 'buoys_datas'.")

            # Conversion JSON → dict si nécessaire
            if isinstance(buoys_datas_raw, str):
                buoys_datas_raw = json.loads(buoys_datas_raw)

            elif isinstance(buoys_datas_raw, pd.DataFrame) and "Station ID" in buoys_datas_raw.columns:
                # Convertir en dictionnaire avec "Station ID" comme clé
                buoys_datas_raw = buoys_datas_raw.set_index("Station ID").to_dict(orient="index")

            # Ajouter au dictionnaire principal directement avec les Station ID comme clés
            buoys_datas.update(buoys_datas_raw)
            buoys_data_count += 1
            total_buoys_rows += len(buoys_datas_raw)  # Compter le nombre de bouées
            print(f"✅ Table 'buoys_datas' chargée avec succès! Nombre de bouées (lignes) : {total_buoys_rows}\n")
        else:
            print("⚠️ Aucun résultat trouvé dans 'buoys_datas'.\n")

    except Exception as e:
        print(f"❌ Erreur lors du chargement de 'buoys_datas': {e}\n")

# Associer les tables marine et meteo en fonction du station_id et récupérer leurs données
for table_set, label, icon, counter, total_rows in [
    (marine_tables, "Marine", "🌊", marine_data_count, total_marine_rows),
    (meteo_tables, "Meteo", "🌧️", meteo_data_count, total_meteo_rows)
]:
    print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
    for table_name in table_set:
        print(f"🔄 Chargement des données pour la table {label} : {table_name}...")

        try:
            station_id = table_name.split("_")[1]

            # Vérifier si la station existe déjà dans buoys_datas, sinon initialiser un dictionnaire
            if station_id not in buoys_datas:
                buoys_datas[station_id] = {}

            # Récupérer les données
            data = fetch_table_data(schema=schema_bronze, conn=conn, table_name=table_name, as_df=True)

            if data is not None:
                print(f"📦 Données récupérées pour la station {station_id} ({label}).")

                if isinstance(data, str):
                    data = pd.DataFrame(json.loads(data))
                elif isinstance(data, dict):
                    data = pd.DataFrame(data)

                # Ajouter les données au dictionnaire de bouées sous la station_id
                buoys_datas[station_id][f"{label} DataFrame"] = data
                counter += 1
                total_rows += len(data)  # Ajouter le nombre de lignes collectées
                print(f"{icon} Données {label} chargées pour la station {station_id}! Nombre de lignes collectées : {len(data)}\n")
            else:
                print(f"⚠️ Aucun résultat trouvé pour la station {station_id} ({label}).\n")

        except Exception as e:
            print(f"❌ Erreur lors du chargement des données {label} pour {table_name} : {e}\n")

# Finalement, afficher un récapitulatif global
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")
print(f"🏆 Chargement des données terminé avec succès !")
print(f"🐋 Total des données bouées chargées : {buoys_data_count} - Nombre de bouées (lignes) : {total_buoys_rows}")
print(f"🌊 Total des données marines chargées : {marine_data_count} - Nombre total de lignes : {total_marine_rows}")
print(f"🌧️ Total des données météorologiques chargées : {meteo_data_count} - Nombre total de lignes : {total_meteo_rows}")
print("━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━")



🔍 Chargement des métadonnées du schéma...
✅ Métadonnées chargées avec succès.

🔢 Nombre total de tables dans le schéma : 79

🌊 Tables marines trouvées : 39
🌧️ Tables météo trouvées : 39
🐋 Tables de bouées trouvées : 1

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔄 Chargement des données de la table 'buoys_datas'...
📦 Données récupérées pour 'buoys_datas'.
✅ Table 'buoys_datas' chargée avec succès! Nombre de bouées (lignes) : 39

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
🔄 Chargement des données pour la table Marine : station_42012_marine_orange beach...
📦 Données récupérées pour la station 42012 (Marine).
🌊 Données Marine chargées pour la station 42012! Nombre de lignes collectées : 6528

🔄 Chargement des données pour la table Marine : station_POTA2_marine_potato point, ak...
📦 Données récupérées pour la station POTA2 (Marine).
🌊 Données Marine chargées pour la station POTA2! Nombre de lignes collectées : 2187

🔄 Chargement des données pour la table Marine : station_46084_marine_cape edgecumbe.

Fuse all the Dataframe in one Final Dataframe

In [36]:
list_silver_merged_df = []  # List to store merged DataFrames
list_failed_dfs = []        # List to store failed DataFrame pairs

number_marine_data = 0
number_meteo_data = 0
number_merged_data = 0
conversions = 0

# Loop through each station in the dictionary
for idx, (station_id, tables) in enumerate(buoys_datas.items()):
    print(f"\n🔄 Processing station {station_id} ({idx + 1}/{len(buoys_datas)})...")

    # Check if the station's buoy data is excluded from normal processing
    # Retrieve weather and marine DataFrames for the station
    df_meteo = buoys_datas[station_id].get("Meteo DataFrame", None)
    df_marine = buoys_datas[station_id].get("Marine DataFrame", None)
    
    # Ensure all necessary data is present
    if df_meteo is None or df_marine is None or buoys_datas[station_id] is None:
        print(f"⚠️ Missing data for station {station_id}. Skipping this station.")
        list_failed_dfs.append(station_id)
        continue
    
    print(f"📦 Data retrieved for station {station_id}:\nMarine: {df_marine.shape[0]} rows\nMeteo: {df_meteo.shape[0]}")
    
    number_marine_data += int(df_marine.shape[0])
    number_meteo_data += int(df_meteo.shape[0])

    # Merge buoy coordinates (Lat/Lon) with marine data
    print("📍 Merging buoy coordinates (Lat/Lon) with marine data...")

    # Directly add the coordinates to df_marine
    df_marine["Lat"] = buoys_datas[station_id]["Lat"]
    df_marine["Lon"] = buoys_datas[station_id]["Lon"]

    # Corrected print statement using single quotes for dictionary keys in the string
    print(f"🌐 Coordinates (Lat/Lon) added for station {station_id}: {df_marine['Lat'].iloc[0]} / {df_marine['Lon'].iloc[0]}.")

    # Convert data types in DataFrames
    print(f"🔄 Converting data types for station {station_id}...")
    df_marine = auto_convert(df_marine)
    df_meteo = auto_convert(df_meteo)
    print(f"✅ Conversion completed for station {station_id}.")
    conversions += 1
    
    # Process and resample marine data
    print(f"🔁 Processing and resampling marine data for station {station_id}...")
    df_marine = process_and_resample(df_marine, column_name='time')
    print(f"✅ Marine data resampled for station {station_id}.")
    
    # Process and resample weather data
    print(f"🔁 Processing and resampling weather data for station {station_id}...")
    df_meteo = process_and_resample(df_meteo, column_name='date')
    print(f"✅ Weather data resampled for station {station_id}.")


    ##### MERGING DATAFRAMES

    # Merge marine and weather DataFrames
    print(f"🔗 Merging marine and weather data for station {station_id}...")
    df_merged = pd.merge(df_marine, df_meteo, on='Datetime', how='inner')
    buoys_datas[str(station_id)]["Merged Dataframe"] = df_merged
    number_merged_data += int(df_merged.shape[0])

    print(f"💾 Merged data added to dictionary for station {station_id}.")
    
# Final merge of all DataFrames
print("🔀 Merging all DataFrames into a final DataFrame...")
# Display the size of DataFrames before final merge

# Extract only the merged DataFrames from each station
dataframes_to_concat = [station_data["Merged Dataframe"] for station_data in buoys_datas.values()]

# Merge the DataFrames
df_final = pd.concat(dataframes_to_concat, ignore_index=True)

# Print some details about the final merge
print(f"📝 Final merged DataFrame size: {df_final.shape}")
print("✅ Final merge completed successfully!")


# Final summary
print("\n⭐🏆 Processing complete!")
print(f"🔢 Total stations processed: {len(buoys_datas)}")
print(f"Marine data rows collected = {number_marine_data}\nMeteo data rows collected = {number_meteo_data}")
if df_final is not None and not df_final.empty:
    print(f"Total Number of merged rows: {number_merged_data}")
    print(f"Final DataFrame rows number: {df_final.shape[0]}")

else:
    print("The DataFrame is either None or empty.")

print(f"🔄 Successful conversions: {conversions}")
print(f"❌ Number of failed stations: {len(list_failed_dfs)}")
if list_failed_dfs:
    print(f"⚠️ Failed stations: {', '.join(list_failed_dfs)}")


🔄 Processing station 41008 (1/39)...
📦 Data retrieved for station 41008:
Marine: 6533 rows
Meteo: 2376
📍 Merging buoy coordinates (Lat/Lon) with marine data...
🌐 Coordinates (Lat/Lon) added for station 41008: 31.40N / 80.87W.
🔄 Converting data types for station 41008...
✅ Conversion completed for station 41008.
🔁 Processing and resampling marine data for station 41008...
Erreur inattendue : agg function failed [how->mean,dtype->object]
✅ Marine data resampled for station 41008.
🔁 Processing and resampling weather data for station 41008...
✅ Weather data resampled for station 41008.
🔗 Merging marine and weather data for station 41008...
💾 Merged data added to dictionary for station 41008.

🔄 Processing station 41044 (2/39)...
📦 Data retrieved for station 41044:
Marine: 6500 rows
Meteo: 2376
📍 Merging buoy coordinates (Lat/Lon) with marine data...
🌐 Coordinates (Lat/Lon) added for station 41044: 21.58N / 58.63W.
🔄 Converting data types for station 41044...
✅ Conversion completed for sta

In [37]:
print(f"{df_final.shape[0]} rows, {df_final.shape[1]} columns\n")
for col in df_final.columns:
    print(f"Col '{col}' : {df_final[col].isna().sum()}")
print(f"\n{df_final.dtypes}")

40927 rows, 37 columns

Col 'Datetime' : 0
Col 'id_x' : 0
Col 'wind_direction' : 1197
Col 'wind_speed' : 368
Col 'wind_gust' : 1453
Col 'wave_height' : 40760
Col 'dominant_wave_period' : 40927
Col 'average_wave_period' : 40760
Col 'dominant_wave_direction' : 40763
Col 'pressure' : 73
Col 'air_temperature' : 6102
Col 'water_temperature' : 11362
Col 'dewpoint' : 10744
Col 'visibility_x' : 40927
Col '3hr_pressure_tendency' : 465
Col 'water_level_above_mean' : 40927
Col 'Station ID' : 0
Col 'Lat' : 0
Col 'Lon' : 0
Col 'id_y' : 0
Col 'temperature_2m' : 0
Col 'relative_humidity_2m' : 0
Col 'dew_point_2m' : 0
Col 'precipitation' : 0
Col 'rain' : 0
Col 'showers' : 0
Col 'pressure_msl' : 0
Col 'surface_pressure' : 0
Col 'cloud_cover' : 0
Col 'cloud_cover_low' : 0
Col 'cloud_cover_mid' : 0
Col 'cloud_cover_high' : 0
Col 'visibility_y' : 0
Col 'wind_speed_10m' : 0
Col 'soil_temperature_0cm' : 0
Col 'soil_moisture_0_to_1cm' : 0
Col 'is_day' : 0

Datetime                   datetime64[ns, UTC+01:00]

In [38]:
col_to_rename={'temperature_2m': 'T°(C°)',  'relative_humidity_2m': 'Relative Humidity (%)',
 'dew_point_2m': 'Dew Point (°C)', 'precipitation': 'Precipitation (mm)',  'pressure_msl':' Sea Level Pressure (hPa)', 
 'cloud_cover_low':'Low Clouds (%)', 'cloud_cover_mid' : 'Middle Clouds (%)',	 'cloud_cover_high' : 'High Clouds (%)', 
 'visibility' : ' Visibility (%)',  'wind_speed_10m' : 'Wind Speed (km/h)',  'wind_direction': 'Wind Direction (°)',
 'wind_speed': 'Wind Speed (km/h)','wind_gust': 'Wind Gusts (km/h)', 'wave_height': 'Wave Height (m)',  'average_wave_period': 'Average Wave Period (s)',
 'dominant_wave_direction': 'Dominant Wave Direction (°)','pressure': 'Pressure (hPA)',
 'air_temperature': 'Air T°','water_temperature': 'Water T°'}
df_final = rename_column(df_final, col_to_rename)

⚠️ Column 'visibility' not found in DataFrame. Skipping renaming.


In [39]:
# Exemple d'utilisation
df_final = handle_null_values(df_final)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_final = df_final.round(2)
print(df_final.columns)
df_final.describe()

Index(['Datetime', 'id_x', 'wind_direction', 'wind_speed', 'wind_gust',
       'pressure', 'air_temperature', 'water_temperature', 'dewpoint',
       '3hr_pressure_tendency', 'Station ID', 'Lat', 'Lon', 'id_y',
       'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation', 'rain', 'showers', 'pressure_msl', 'surface_pressure',
       'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'visibility_y', 'wind_speed_10m', 'soil_temperature_0cm',
       'soil_moisture_0_to_1cm', 'is_day'],
      dtype='object')


,id_x,wind_direction,wind_speed,pressure,3hr_pressure_tendency,id_y,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,...,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility_y,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
count,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,...,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000
mean,2824.777335,173.969262,6.914509,1013.174736,-0.017827,1680.594180,0.482064,73.833899,-3.921916,0.051186,...,990.151990,56.155448,42.575781,27.943240,17.722262,22782.024092,18.180529,3.228291,0.315092,0.464143
std,1971.270486,102.467301,3.453281,10.837817,1.659010,317.665529,6.423901,14.000327,6.004250,0.246077,...,10.176774,43.306135,43.816652,40.830865,34.752150,9940.821849,7.819397,4.811258,0.021823,0.498719
min,1.000000,10.000000,0.000000,939.900000,-12.100000,1129.000000,-13.240000,32.000000,-19.050000,0.000000,...,962.760000,0.000000,0.000000,0.000000,0.000000,100.000000,1.140000,-2.260000,0.260000,0.000000
25%,971.000000,90.000000,4.000000,1009.100000,-1.000000,1405.000000,-3.590000,65.000000,-7.550000,0.000000,...,983.700000,5.000000,1.000000,0.000000,0.000000,16800.000000,12.590000,-0.360000,0.300000,0.000000
50%,2588.000000,160.000000,7.000000,1015.600000,0.000000,1682.000000,-0.490000,74.000000,-4.100000,0.000000,...,989.180000,69.000000,21.000000,0.000000,0.000000,22300.000000,17.380000,1.440000,0.320000,0.000000
75%,4550.000000,270.000000,9.000000,1019.900000,1.000000,1958.000000,3.410000,84.000000,-0.350000,0.000000,...,998.170000,100.000000,100.000000,69.000000,5.000000,28000.000000,23.400000,5.740000,0.330000,1.000000
max,6552.000000,360.000000,25.200000,1041.100000,10.900000,2223.000000,20.310000,100.000000,12.880000,2.800000,...,1014.000000,100.000000,100.000000,100.000000,100.000000,60600.000000,39.710000,18.240000,0.390000,1.000000


In [ ]:
df_dew_point = df_final[["dew_point_2m", "dewpoint"]]
df_dew_point.isna().sum()
# Have to Drop dew_point as its not trustworthy enough

Colonne 'id_x' Supprimée
Colonne 'soil_temperature_0cm' Supprimée
Colonne 'id_y' Supprimée


In [ ]:
df_final = drop_columns_if_exist(df=df_final, columns_to_drop=
                      ["id_x","soil_temperature_0cm", "id_y",
                       ])

Index(['Datetime', 'id_x', 'wind_direction', 'wind_speed', 'wind_gust',
       'pressure', 'air_temperature', 'water_temperature', 'dewpoint',
       '3hr_pressure_tendency', 'Station ID', 'Lat', 'Lon', 'id_y',
       'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
       'precipitation', 'rain', 'showers', 'pressure_msl', 'surface_pressure',
       'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
       'visibility_y', 'wind_speed_10m', 'soil_temperature_0cm',
       'soil_moisture_0_to_1cm', 'is_day'],
      dtype='object')

In [ ]:
df_final.describe()

,id_x,wind_direction,wind_speed,pressure,3hr_pressure_tendency,id_y,temperature_2m,relative_humidity_2m,dew_point_2m,precipitation,...,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,visibility_y,wind_speed_10m,soil_temperature_0cm,soil_moisture_0_to_1cm,is_day
count,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,...,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000,40927.000000
mean,2824.777335,173.969262,6.914509,1013.174736,-0.017827,1680.594180,0.482064,73.833899,-3.921916,0.051186,...,990.151990,56.155448,42.575781,27.943240,17.722262,22782.024092,18.180529,3.228291,0.315092,0.464143
std,1971.270486,102.467301,3.453281,10.837817,1.659010,317.665529,6.423901,14.000327,6.004250,0.246077,...,10.176774,43.306135,43.816652,40.830865,34.752150,9940.821849,7.819397,4.811258,0.021823,0.498719
min,1.000000,10.000000,0.000000,939.900000,-12.100000,1129.000000,-13.240000,32.000000,-19.050000,0.000000,...,962.760000,0.000000,0.000000,0.000000,0.000000,100.000000,1.140000,-2.260000,0.260000,0.000000
25%,971.000000,90.000000,4.000000,1009.100000,-1.000000,1405.000000,-3.590000,65.000000,-7.550000,0.000000,...,983.700000,5.000000,1.000000,0.000000,0.000000,16800.000000,12.590000,-0.360000,0.300000,0.000000
50%,2588.000000,160.000000,7.000000,1015.600000,0.000000,1682.000000,-0.490000,74.000000,-4.100000,0.000000,...,989.180000,69.000000,21.000000,0.000000,0.000000,22300.000000,17.380000,1.440000,0.320000,0.000000
75%,4550.000000,270.000000,9.000000,1019.900000,1.000000,1958.000000,3.410000,84.000000,-0.350000,0.000000,...,998.170000,100.000000,100.000000,69.000000,5.000000,28000.000000,23.400000,5.740000,0.330000,1.000000
max,6552.000000,360.000000,25.200000,1041.100000,10.900000,2223.000000,20.310000,100.000000,12.880000,2.800000,...,1014.000000,100.000000,100.000000,100.000000,100.000000,60600.000000,39.710000,18.240000,0.390000,1.000000


In [ ]:
# def explore_dict_keys(d, parent_key='', sep='_'):
#     """
#     Explore un dictionnaire récursivement pour obtenir toutes les clés, y compris les sous-clés,
#     mais ne retourne pas les valeurs finales.

#     :param d: Le dictionnaire à explorer
#     :param parent_key: La clé parent qui est utilisée pour concaténer les sous-clés
#     :param sep: Le séparateur utilisé pour concaténer les clés (par défaut '_')
#     :return: Une liste des clés (et sous-clés)
#     """
#     keys = []
#     for k, v in d.items():
#         new_key = f"{parent_key}{sep}{k}" if parent_key else k
#         if isinstance(v, dict):  # Si la valeur est un dictionnaire, on explore récursivement
#             keys.append(new_key)  # Ajouter la clé, mais ne pas inclure la valeur
#             keys.extend(explore_dict_keys(v, new_key, sep=sep))  # Continuer l'exploration
#         else:
#             keys.append(new_key)  # Ajouter la clé finale
#     return keys

In [ ]:
# def find_key_path(d, target_key, path=[]):
#     """
#     Recherche récursive d'une clé dans un dictionnaire et retourne son chemin.
#     :param d: dictionnaire
#     :param target_key: clé recherchée
#     :param path: liste pour stocker le chemin jusqu'à la clé
#     :return: chemin sous forme de liste
#     """
#     if isinstance(d, dict):  # Si le dictionnaire est encore imbriqué
#         for key, value in d.items():
#             new_path = path + [key]
#             if key == target_key:
#                 return new_path
#             elif isinstance(value, dict):
#                 result = find_key_path(value, target_key, new_path)
#                 if result:  # Si la clé est trouvée, retourner le chemin
#                     return result
#     return None  # Retourne None si la clé n'a pas été trouvée



# # Recherche du chemin pour la clé 'marine_data'
# path = find_key_path(table_dict, "Marine Dataframe")
# print(path)


Auto_convert Test

In [ ]:
# for idx, (buoy_id, tables) in enumerate(table_dict.items()):  # Utilisation de .items() pour obtenir (clé, valeur)
#     if isinstance(tables, dict):
#         if idx == 1:  # Vérifier si l'index est égal à 1

Counting Rows of all Dataframes in total